  ## Логистическая регрессия, Регуляризация, Наивный Байес, K-ближайших соседей, Кросс-валидация и Подбор гиперпараметров
  <hr>

  ```
  План лабораторной работы

  1. Логистическая регрессия
  2. Данные для задачи классификации (Data for Classification Task)
  3. Работа с несбалансированными данными (Dealing with data imbalance)
  4. Метрики оценки задачи классификации (Classification task evaluation metrics)
  5. Регуляризация: Lasso и Ridge
  6. Наивный Байес
  7. K-ближайших соседей (KNN)
  8. Кросс-валидация
  9. Подбор гиперпараметров
  ```

  <hr>

 ## 1. Логистическая регрессия

 Логистическая регрессия - это статистическая модель, используемая для задач бинарной (и мультиклассовой) классификации.

 Она оценивает вероятность принадлежности к определенному классу с помощью логистической (сигмоидной) функции.



 $$\hat p(x) = \frac{e^{\beta_0 + \beta_1 x}}{1+e^{\beta_0 + \beta_1 x}}$$



 <br>

 Функция потерь (Log Loss) используется для оценки производительности модели. Она штрафует модели за неправильные предсказания, особенно когда модель уверена в неправильном ответе.

 $$L(\hat{p}(x_i), y_i) = -y_i * \log (\hat{p}(x_i)) - (1 - y_i) * \log (1 -\hat{p}(x_i))$$



 <br>

 Это можно переписать как:

 $$L(\hat{p}(x_i), y_i) = \left\{\begin{matrix}

 \ - log (\hat{p}(x_i)), & y_i=1\\

 \ - log (1 -\hat{p}(x_i)), & y_i=0

 \end{matrix}\right.$$



 <br>

 Для получения конечного предсказания (0 или 1) используется порог (threshold), обычно 0.5.

 $$\hat y = \left\{\begin{matrix}

 1 && \hat p(x) > threshold\\

 0 && otherwise

 \end{matrix}\right.$$



 1. Как выбрать порог (threshold)? Это важный вопрос, зависящий от баланса между точностью (precision) и полнотой (recall) для конкретной задачи.

    Подробнее: [User Guide: Adjusting the prediction threshold](https://scikit-learn.org/stable/modules/calibration.html#adjusting-the-prediction-threshold)



 <br>

 Давайте посмотрим, как форма $\hat p(x)$ зависит от её параметров:

 ### 1.1 Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline


 ### 1.2 Построение сигмоидальной функции

 Сигмоида - это S-образная кривая, которая отображает любое действительное число в значение от 0 до 1.

 Параметр b0 (смещение) сдвигает кривую вдоль оси x, а b1 (коэффициент при x) определяет крутизну кривой.

 Подробнее о сигмоиде: [Wikipedia: Sigmoid function](https://en.wikipedia.org/wiki/Sigmoid_function)

In [ ]:
x = np.arange(-10, 10, 0.01)


def plot(b0, b1):
    p = np.exp(b0 + b1 * x) / (1 + np.exp(b0 + b1 * x))
    label = "b0 = {}, b1 = {}".format(b0, b1)
    plt.plot(x, p, label=label)


plot(0, 1)
plot(0, 2)
plot(0, 3)
plot(5, 1)
plt.legend()
plt.title("Сигмоидальная функция")
plt.show()


 Что контролируют параметры b0, b1?

 ### 1.3 Построение функции потерь

 График показывает, как велика потеря (loss) в зависимости от предсказанной вероятности $\hat p(x)$ для истинных меток y=1 и y=0.

 Обратите внимание, что потеря стремится к бесконечности, когда модель ошибается с высокой уверенностью.

In [ ]:
x = np.arange(0.001, 1, 0.001)
y1 = -np.log(x)
y0 = -np.log(1 - x)

plt.plot(x, y1, label="если y = 1")
plt.plot(x, y0, label="если y = 0")
plt.legend()
plt.show()


 В чём преимущества использования логарифмов в функции потерь? <br><br>

 ## 2. Данные для задачи классификации

 В задачах классификации целевая переменная (y) является категориальной (например, 'здоров', 'болен'), в то время как в задачах регрессии она непрерывна (например, цена дома).

 Проведём анализ того, какие типы людей, скорее всего, выжили.

 В частности, мы попросим вас применить инструменты машинного обучения для предсказания того, какие пассажиры выжили в трагедии Титаника.

 ### 2.1 Загрузка данных

In [ ]:
titanic_df = pd.read_csv("titanic.csv")
titanic_df.head()


 ### 2.2 Разведочный анализ данных (EDA)



 Подход к анализу наборов данных для суммирования их основных характеристик, часто с использованием графической статистики и других методов визуализации данных (например, matplotlib, plot распределения ..)<br>

 Сегодня мы попробуем инструмент под названием [ydata profiling](https://github.com/ydataai/ydata-profiling  ). 
 **Примечание:** Всё, что делает pandas profilling, можно легко достичь с помощью pandas, matplotlib и numpy

In [ ]:
from ydata_profiling import ProfileReport

## Создать отчет о профиле данных
profile = ProfileReport(
    titanic_df, title="Отчет о профиле"
).to_notebook_iframe()


 ### 2.3 Предварительная обработка данных

 1. Как бороться с пропущенными значениями? Используются стратегии импутации (mean, median, mode, константа, модели).

    Подробнее: [User Guide: Imputation](https://scikit-learn.org/stable/modules/impute.html)

 1. Как бороться с категориальными данными? Применяются методы кодирования (One-Hot, Label, Ordinal и др.).

    Подробнее: [User Guide: Encoding categorical features](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features)

 1. Нужно ли масштабирование данных? Для логистической регрессии масштабирование может улучшить сходимость и устойчивость.

    Подробнее: [User Guide: Scaling features](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaling)



 **ЗАДАНИЕ:**

 1. Разделить данные на обучающую и тестовую выборки

 2. Заполнить пропущенные значения

 3. Использовать MinMaxScaler для масштабирования признаков

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Выбрать признаки
titanic_df.drop(["name"], axis=1, inplace=True)

# TODO: Разделить данные на обучающую и тестовую выборки


# TODO: Заполнить пропущенные значения


# one-hot-encode категориальных признаков
def ohe_new_features(df, features_name, encoder):
    new_feats = encoder.transform(df[features_name])
    # Создаем имена для новых столбцов
    new_columns = [
        f"{features_name[0]}_{i}" for i in range(new_feats.shape[1])
    ]
    new_cols = pd.DataFrame(new_feats, columns=new_columns, dtype=int)
    new_df = pd.concat([df.reset_index(drop=True), new_cols], axis=1)
    new_df.drop(features_name, axis=1, inplace=True)
    return new_df


encoder = OneHotEncoder(sparse_output=False, drop="first")
f_names = ["sex", "embarked"]
encoder.fit(x_train[f_names])
x_train = ohe_new_features(x_train, f_names, encoder)
x_test = ohe_new_features(x_test, f_names, encoder)

# TODO: масштабирование признаков с использованием MinMaxScaler


 ### 2.4 Построение, обучение и тестирование модели



 Теперь мы готовы увидеть логистическую регрессию на практике.

 Подробнее о логистической регрессии в scikit-learn: [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)



 ### Задание.

 1. Обучить логистическую регрессию

 1. Вывести метрики точности (Accuracy), полноты (Precision) и F-меры (Recall) на тестовом наборе.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# TODO: обучить логистическую регрессию
clf = None
y_test_pred = None

# TODO: вычислить метрики


 ###  2.5 Интерпретация результатов предсказания и измерение производительности модели



 1. изучение коэффициентов логистической регрессии

 2. порог предсказания



 ### Задание :

 1. Вычислить значения точности (Accuracy), полноты (Precision) и F-меры (Recall) для каждого из заданных пороговых значений и построить их графики.

 Подробнее о метриках можно почитать [здесь](https://scikit-learn.org/stable/modules/model_evaluation.html#from-binary-to-multiclass-and-multilabel).

In [ ]:
# Коэффициенты логистической регрессии
print("----Коэффициенты логистической регрессии----")
print(*[a for a in zip(list(x_train.columns), clf.coef_[0])], sep="\n")


In [ ]:
# TODO: вычислить метрики для каждого порога выше и построить результат, как показано ниже.
thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
pred_proba = clf.predict_proba(x_test)

results = [[],[],[]]
for i in thresholds:
    #TODO: Вычислить accuracy_score, precision_score & recall_score

plt.plot(thresholds, results[0], label = 'accuracy')
plt.plot(thresholds, results[1], label = 'precision')
plt.plot(thresholds, results[2], label = 'recall')
plt.title('Выбор порога')
plt.xlabel('threshold')
plt.ylabel('score')
plt.legend()
plt.grid()


 ### Матрица ошибок (Confusion matrix)

 Матрица ошибок предоставляет подробную информацию о производительности классификатора, показывая, какие классы и как часто путаются.

 Подробнее: [User Guide: Confusion matrix](https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix)

 <table><tr><td>

 <img align='center' src='https://static.packt-cdn.com/products/9781838555078/graphics/C13314_06_05.jpg  ' style='width: 350px;'>

 </td><td>

 <img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_confusion_matrix_002.png  ' style='width: 400px;'></td></tr></table>

 <br>





 ### Задание:

 1. Реализовать метод матрицы ошибок с нуля с использованием списков python и numpy

In [ ]:
def calc_confusion_matrix(true_labels, pred_labels):
    """Вычислить матрицу ошибок для оценки точности классификации

    Параметры
    ----------
    true_labels : array-like формы (n_samples,)
        Истинные (правильные) целевые значения.
    pred_labels : array-like формы (n_samples,)
        Оцененные цели, возвращенные классификатором.
    """
    # TODO : Реализовать
    return None


In [ ]:
## вычисление матрицы ошибок
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
print("Матрица ошибок для теста 1")
print(calc_confusion_matrix(y_true, y_pred))

titanic_cm = calc_confusion_matrix(y_test, clf.predict(x_test))
print("Матрица ошибок для Титаника:\n", titanic_cm, "\n")


In [ ]:
# Запустите этот блок кода, чтобы увидеть, как должен выглядеть ваш результат
from sklearn.metrics import confusion_matrix

print(
    "sklearn Матрица ошибок для тестового набора:\n",
    confusion_matrix(y_true, y_pred),
)
print(
    "sklearn Матрица ошибок для Титаника:\n",
    confusion_matrix(y_test, clf.predict(x_test)),
    "\n",
)


 ## 5. Регуляризация: Lasso и Ridge

 Обе модели являются регуляризованными формами линейной регрессии.

 Lasso (L1-регуляризация) может обнулять коэффициенты, тем самым выполняя отбор признаков.

 Ridge (L2-регуляризация) уменьшает величину коэффициентов, но не обнуляет их.

 Подробнее: [User Guide: Linear Models](https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression-and-classification)

 ### Проблемы:

 1. Когда использовать Lasso?



 2. Когда использовать Ridge?



 3. Так как трудно определить влияние параметров, как мы можем решить, какую регуляризацию использовать? и определить значение lambda (alpha в sklearn)?



 ### 5.1 Загрузка датасета Boston

 Значения цен на жилье в пригородах Бостона.

In [ ]:
# Используем альтернативную функцию для загрузки датасета Boston
import pandas as pd
import numpy as np


def load_boston():
    header = [
        "CRIM",
        "ZN",
        "INDUS",
        "CHAS",
        "NOX",
        "RM",
        "AGE",
        "DIS",
        "RAD",
        "TAX",
        "PTRATIO",
        "B",
        "LSTAT",
        "MEDV",
    ]
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    target = np.expand_dims(raw_df.values[1::2, 2], 1)
    df = pd.DataFrame(np.hstack([data, target]), columns=header)
    return df


boston_df = load_boston()
X = boston_df.drop("MEDV", axis=1).values
y = boston_df["MEDV"].values

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123
)
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=1 / 8, random_state=123
)


 ### 5.2 Обучение моделей Lasso и Ridge

 Подробнее о Lasso: [sklearn.linear_model.Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

 Подробнее о Ridge: [sklearn.linear_model.Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)



 ### Задание:

 1. Обучить две модели: Lasso и Ridge - со значением alpha по умолчанию.

 2. Затем вывести их коэффициенты и обратить внимание на разницу.

In [ ]:
from sklearn.linear_model import Lasso, Ridge

lasso = None  # TODO: инициализировать Lasso
ridge = None  # TODO: инициализировать Ridge
lasso.fit(x_train, y_train)
ridge.fit(x_train, y_train)

# TODO: вывести коэффициенты моделей


 ### Задание:

 1. Давайте попробуем разные значения для alpha для регрессора Lasso и построим график потерь на валидации.

In [ ]:
import matplotlib.pylab as plt
import numpy as np
from sklearn.metrics import mean_squared_error
%matplotlib inline

alphas = [2.2, 2, 1.5, 1.3, 1.2, 1.1, 1, 0.3, 0.1]
losses = []
for alpha in alphas:
    # TODO:
    # Написать (5 строк): создать регрессор Lasso с указанным значением alpha.
    # Обучить его на обучающей выборке, затем получить предсказание на валидационной выборке (x_val).
    # вычислить среднеквадратичную ошибку, затем добавить её в массив losses

plt.plot(alphas, losses)
plt.title("Выбор значения alpha для Lasso")
plt.xlabel("alpha")
plt.ylabel("Среднеквадратичная ошибка")
plt.show()

best_alpha = alphas[np.argmin(losses)]
print("Лучшее значение alpha:", best_alpha)


 Измерение потерь на тестовом наборе с регрессором Lasso с лучшим alpha.

In [ ]:
lasso = Lasso(best_alpha)
lasso.fit(x_train, y_train)
y_pred = lasso.predict(x_test)
print(
    "Среднеквадратичная ошибка на тестовом наборе:",
    mean_squared_error(y_test, y_pred),
)


 ## 6. Наивный Байес

 Мы будем использовать гауссовский наивный Байес (`GaussianNB`), который, исходя из предположения, работает с непрерывными признаками как с гауссовыми переменными для вычисления их вероятности.

 $$P(x_i|y) = \frac{1}{\sqrt{2\pi\sigma_y^2}}exp(-\frac{(x_i - \mu_y)^2}{2\sigma_y^2})$$

 Где $\mu_y$ и $\sigma_y^2$ - среднее и дисперсия признака $i$ для класса $y$.

 Подробнее: [User Guide: Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)

 Примечание: Различные классификаторы наивного Байеса в основном различаются предположениями, которые они делают относительно распределения $P(x_i|y)$.

 ___

 Каковы плюсы и минусы классификатора наивного Байеса?



 ___

 ### 6.1 Загрузка датасета

 Датасет из 10 классов цифр, состоящий из изображений 8x8 пикселей. Подходит для задачи классификации!

In [ ]:
from sklearn.datasets import load_digits

X, y = load_digits(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# Мы покажем, почему не разделили валидационный набор.


In [ ]:
x_train.shape


 ### 6.2 Построение, обучение и тестирование модели

 Подробнее о GaussianNB: [sklearn.naive_bayes.GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)

 Давайте обучим модель наивного Байеса и проверим точность на тестовом наборе.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gauss_nb = GaussianNB()
gauss_nb.fit(x_train, y_train)
y_pred = gauss_nb.predict(x_test)
print(accuracy_score(y_test, y_pred))


 #### Задание : Вычислить простую матрицу ошибок и нормализованную матрицу ошибок для оценки производительности модели

 **Примечание:** См. [`sklearn.metrics.plot_confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html) (функция `plot_confusion_matrix` устарела, используйте `ConfusionMatrixDisplay.from_estimator`)

In [ ]:
# TODO: простая матрица ошибок
from sklearn.metrics import ConfusionMatrixDisplay

# Используйте ConfusionMatrixDisplay.from_estimator(clf, x_test, y_test) для простой матрицы


In [ ]:
# TODO: нормализованная матрица ошибок
# Используйте normalize='true' в from_estimator или from_predictions




 ## 7. Классификатор k-ближайших соседей (KNN)

 KNN - это непараметрический метод, основанный на схожести. Прогноз делается на основе k ближайших соседей.

 Подробнее: [User Guide: Nearest Neighbors](https://scikit-learn.org/stable/modules/neighbors.html)

 1. Каковы плюсы и минусы KNN?



     <span style="color:blue">





 2. Чтобы увеличить дисперсию модели KNN, нужно увеличить или уменьшить K?



 ### 7.1 Загрузка и предварительная обработка данных

 Подробнее о StandardScaler: [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

 Давайте сделаем то же самое с классификатором KNN.

 Почему нужно масштабировать признаки? KNN чувствителен к масштабу признаков, так как использует расстояния.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
print(accuracy_score(y_test, y_pred))


 #### Как выбрать оптимальный $k$ ?

 Давайте настроим гиперпараметр $n\_neighbors$ в объекте классификатора KNN, используя кросс-валидацию.



 ## 8. Кросс-валидация

 Кросс-валидация приходит как альтернатива разделению на валидационный набор.

 Подробнее: [User Guide: Cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html)

 Примечание: поэтому мы не создавали валидационный набор.

In [ ]:
from sklearn.model_selection import cross_val_score
import matplotlib.pylab as plt
import numpy as np

%matplotlib inline

Ks = list(range(1, 20))
cv_scores = []
for K in Ks:
    knn = KNeighborsClassifier(n_neighbors=K)
    scores = cross_val_score(knn, x_train, y_train, cv=7, scoring="accuracy")
    avg_score = np.mean(scores)
    cv_scores.append(avg_score)

plt.plot(Ks, cv_scores)
plt.show()
print(cv_scores)
print(Ks[np.argmax(cv_scores)])


 В классификаторе KNN есть несколько гиперпараметров для настройки, настройка их по одному - это трудоемкий подход.

 Давайте попробуем лучший подход, называемый GridSearchCV.

 ## 9. Подбор гиперпараметров

 Подбор гиперпараметров - это процесс выбора набора оптимальных гиперпараметров для алгоритма машинного обучения.

 Подробнее: [User Guide: Tuning the hyper-parameters of an estimator](https://scikit-learn.org/stable/modules/grid_search.html)

 Стратегии настройки:

 1. Поиск по сетке (Grid Search): перебор всех комбинаций. Подробнее: [sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

 1. Случайный поиск (Random Search): случайный выбор параметров из заданного пространства поиска. Подробнее: [sklearn.model_selection.RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)



 Задание:

 Использовать gridsearch для настройки 3 гиперпараметров:



 1. $n\_neighbors$: `[1, 2, . . ., 10]`

 2. $weights$: `['uniform', 'distance']`

 3. $metric$: `['euclidean', 'manhattan', 'chebyshev', 'cosine']`



 См. этот [ссылка](https://scikit-learn.org/stable/modules/grid_search.html) для справки.



 Затем измерить точность на тестовом наборе.

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    "n_neighbors": list(range(1, 11)),
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan", "chebyshev", "cosine"],
}

grid_search_clf = GridSearchCV(
    estimator=KNeighborsClassifier(),
    cv=7,
    scoring="accuracy",
    param_grid=param_grid,
)

grid_search_clf.fit(x_train, y_train)
means = grid_search_clf.cv_results_["mean_test_score"]
stds = grid_search_clf.cv_results_["std_test_score"]
for mean, std, params in zip(
    means, stds, grid_search_clf.cv_results_["params"]
):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print()
print("Найденный набор лучших параметров:")
print()
print(grid_search_clf.best_params_)

y_pred = grid_search_clf.predict(x_test)
print(accuracy_score(y_test, y_pred))


 ### 9.1 Задание :

 Использовать случайный поиск для настройки 3 гиперпараметров:



 1. $n\_neighbors$: `[1, 2, . . ., 10]`

 2. $weights$: `['uniform', 'distance']`

 3. $metric$: `['euclidean', 'manhattan', 'chebyshev', 'cosine']`



 См. этот [ссылка](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) для справки.



 Затем измерить точность на тестовом наборе.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Измените следующие строки, чтобы запустить RandomizedSearchCV с cv=5
param_grid = {
    "n_neighbors": list(range(1, 11)),
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan", "chebyshev", "cosine"],
}  # TODO : Определить пространство поиска гиперпараметров

randomized_search_clf = RandomizedSearchCV(
    estimator=KNeighborsClassifier(),
    cv=5,
    scoring="accuracy",
    param_distributions=param_grid,
    n_iter=10,
    random_state=42,
)  # TODO : Определить класс случайного поиска из sklearn
randomized_search_clf.fit(
    x_train, y_train
)  # random заменен на randomized_search_clf

print("Лучший результат: ", randomized_search_clf.best_score_)
print("Лучшие параметры: ", randomized_search_clf.best_params_)


In [ ]:
y_pred = randomized_search_clf.predict(x_test)
print(f"Тест на тестовом наборе : {accuracy_score(y_test, y_pred):.4}")

# TODO: Добавьте сравнение результатов GridSearch и RandomizedSearch

# TODO: Добавьте HalvingGridSearchCV или HalvingRandomSearchCV для сравнения скорости и качества
